<a href="https://colab.research.google.com/github/iampushpdeep/gsoc_task_ml4sci_faseroh/blob/main/Taylorseries_Data_LSTM_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FASEROH Task - 1 Generate Taylor Series Data
## Pushpdeep Singh(pushpdeep30@gmail.com)
### (I am using taylor series data around zero) (Also, many polynomials fucntions have taylor series equal to themselves so I ignore them so our model doesn't learn y=x function)(This dataset file is uploaded on my github repo - )




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
import functools
from random import choices
import time
import sympy as sp
x = sp.Symbol('x')
import numpy as np

# leaves and binary operators
leaves = ["x", "-5", "-4", "-3", "-2", "-1", "1", "2", "3", "4", "5"]
p_leaf = [2/3, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30]
binary_operators = ["+", "-", "*", "/"]
unary_operators = ["exp", "sqrt", "sin", "cos", "tan", "sinh", "cosh", "tanh", "asin", "acos", "atan", "asinh", "acosh", "atanh"]


@functools.lru_cache(128)
def unary_binary_subtrees(e, n):
    """ calculate D(e, n), helper for distribution_k_a """
    if e==0:
        return 0
    elif n==0:
        return 1
    else:
        return unary_binary_subtrees(e-1, n) + unary_binary_subtrees(e, n-1) + unary_binary_subtrees(e+1, n-1)


def distribution_k_a(e, n):
    """ distribution over position k(e, n) """
    population = []
    weights = []
    for k in range(e):
        population.append( (k, 1))
        e_n = unary_binary_subtrees(e, n)
        weights.append(unary_binary_subtrees(e-k, n-1)/e_n)
        population.append((k, 2))
        weights.append(unary_binary_subtrees(e-k+1, n-1)/e_n)
    return population, weights


class NodeBinary:
    operator = True
    binary = True
    operand = False
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right


class NodeUnary:
    operator = True
    binary = False
    operand = False
    def __init__(self, data, middle=None):
        self.data = data
        self.middle = middle


class Leaf:
    operator = False
    binary = False
    operand = True
    def __init__(self, data):
        self.data = data


def random_binary_trees(n):
    """
    Generates a random binary tree
    see Appendix C:
        n = num nodes
        k = position
        a = arity (unary or binary operator)
        e = number of empty nodes
    """
    population, weights = distribution_k_a(1, n)
    k, a = choices(population, weights)[0]

    #  select root node
    if a == 1:
        operator = choices(unary_operators)[0]
        node = NodeUnary(operator)
        empty_node = [(node, 'middle')]
        e = 1
    else:
        operator = choices(binary_operators)[0]
        node = NodeBinary(operator)
        empty_node = [(node, 'left'), (node, 'right')]
        e = 2

    #  making the tree
    n = n - 1
    while n > 0:
        population, weights = distribution_k_a(e, n)
        k, a = choices(population, weights)[0]
        i = 0
        new_empty_node = []
        for ele in empty_node:
            if i < k:
                leave = choices(leaves, p_leaf)[0]
                value = Leaf(leave)
                setattr(ele[0], ele[1], value)
            elif i == k:
                if a == 1:
                    operator = choices(unary_operators)[0]
                    value = NodeUnary(operator)
                    setattr(ele[0], ele[1], value)
                    if ele[1] == 'left':
                        new_ele = ele[0].left
                    elif ele[1] == 'middle':
                        new_ele = ele[0].middle
                    else:
                        new_ele = ele[0].right
                    new_empty_node = [(new_ele, 'middle')]
                    e = e - k
                else:
                    operator = choices(binary_operators)[0]
                    value = NodeBinary(operator)
                    setattr(ele[0], ele[1], value)
                    if ele[1] == 'left':
                        new_ele = ele[0].left
                    elif ele[1] == 'middle':
                        new_ele = ele[0].middle
                    else:
                        new_ele = ele[0].right
                    new_empty_node = [(new_ele, 'left'), (new_ele, 'right')]
                    e = e - k + 1
            else:
                new_empty_node.append(ele)
            i += 1
        empty_node = new_empty_node
        n = n - 1

    if len(empty_node) != 0:
        for ele in empty_node:
            leave = choices(leaves, p_leaf)[0]
            value = Leaf(leave)
            setattr(ele[0], ele[1], value)

    return node


def traverse_unary_binary_prefix(root, seq=None, verbose=False):
    """ traverses a binary expression tree and generates prefix notation """
    if not seq:
        seq = []

    if verbose:
        print(root.data)

    seq.append(root.data)

    if root.binary:
        if root.left.operator:
            traverse_unary_binary_prefix(root.left, seq)
        else:
            if verbose:
                print(root.left.data)
            seq.append(root.left.data)

        if root.right.operator:
            traverse_unary_binary_prefix(root.right, seq)
        else:
            if verbose:
                print(root.right.data)
            seq.append(root.right.data)
    else:
        if root.middle.operator:
            traverse_unary_binary_prefix(root.middle, seq)
        else:
            if verbose:
                print(root.middle.data)
            seq.append(root.middle.data)

    return seq

In [ ]:
import random

from sympy import *



OPERATORS = set(['+', '-', '*', '/', '(', ')', 'pow'])

UNARY_OPERATORS = set(["exp", "sqrt", "sin", "cos", "tan", "asin", "acos", "atan", "sinh", "cosh", "tanh", "asinh", "acosh", "atanh"])

operator_class = ["<class 'sympy.core.add.Add'>", "<class 'sympy.core.mul.Mul'>", "<class 'sympy.core.power.Pow'>", "exp", "sin", "cos", "tan", "asin", "acos", "atan", "sinh", "cosh", "tanh", "asinh", "acosh", "atanh", "Abs"]

operator_dict = {"<class 'sympy.core.add.Add'>": "+", "<class 'sympy.core.mul.Mul'>": "*", "<class 'sympy.core.power.Pow'>": "pow", "exp": "exp", "sin": "sin", "cos": "cos", "tan": "tan", "asin": "asin", "acos": "acos", "atan": "atan", "sinh": "sinh", "cosh": "cosh", "tanh": "tanh", "asinh": "asinh", "acosh": "acosh", "atanh": "atanh", "Abs": "abs"}

rational_number = ["<class 'sympy.core.numbers.Rational'>", "<class 'sympy.core.numbers.Half'>"]

exp1_number = ["<class 'sympy.core.numbers.Exp1'>"]

invalid_function = ["<class 'sympy.core.numbers.ImaginaryUnit'>", "<class 'sympy.core.numbers.ComplexInfinity'>", "<class 'sympy.core.numbers.NegativeInfinity'>", "<class 'sympy.core.numbers.Infinity'>", "<class 'sympy.core.numbers.NaN'>"]

invalid_expression = ["zoo", "oo", "I", "-oo"]

#x = symbols('x')
def infix_to_prefix(expr, seq=None):
    if not seq:
        seq = []
    s1 = str(expr)
    s2 = str(expr.func)
    if s1 in invalid_expression or s2 in invalid_function:
        return False
    length = len(expr.args)
    if s2 in operator_class:
        op = operator_dict[s2]
        seq.append(op)
    elif s2 in rational_number:
        s1 = s1.split("/")
        seq.append("/")
        seq.append(s1[0])
        seq.append(s1[1])
    elif s2 in exp1_number:
        seq.append("exp")
        seq.append("1")
    else:
        seq.append(s1)

    for i in range(length):
        if i!=0 and i!=(length-1):
            seq.append(op)
        arg = expr.args[i]
        feedback = infix_to_prefix(arg, seq)
        if feedback==False:
            return False
    return seq

### PREFIX ===> INFIX ###
'''
Scan the formula reversely
1) When the token is an operand, push into stack
2) When the token is an operator, pop out 2 numbers from stack, merge them and push back to the stack
'''
def prefix_to_infix(formula):
    stack = []
    for ch in reversed(formula):
        if ch not in OPERATORS and ch not in UNARY_OPERATORS:
            stack.append(ch)
        else:
            if ch in OPERATORS:
                a = stack.pop()
                b = stack.pop()
                exp = "(" + " " + a + " " + ch + " " + b + " " + ")"  
            else:
                a = stack.pop()
                exp = ch + " " + "(" + " " +  a + " " +")"
            stack.append(exp)
    return stack[-1].split()




In [ ]:
import random
import time

from sympy import Symbol, diff, simplify

In [ ]:

_FINISH = False
x = Symbol('x', real=True) # TODO


def simplify_timeout(expr):
    from multiprocessing import Process, Manager

    result = None
    def f(d, expr):
        from sympy import S; S.Half
        try:
            d['result'] = simplify(expr)
        except:
            d['result'] = "Error"

    with Manager() as manager:
        d = manager.dict()
        p = Process(target=f, args=(d, expr))
        p.start()
        p.join(timeout=3)
        if p.is_alive():
            p.terminate()
        else:
            result = d["result"]
    if result==None:
        return expr
    elif str(type(result))=="<class 'sympy.calculus.util.AccumulationBounds'>":
        return "Error"
    else:
        return result

def series_timeout(ff):
  from multiprocessing import Process, Manager
  expr = None
  def f(d, ff):
      from sympy import S; S.Half
      try:
          d['expr'] = ff.series(x, 0, 5).removeO()
      except:
          d['expr'] = "Error"

  with Manager() as manager:
      d = manager.dict()
      p = Process(target=f, args=(d, ff))
      p.start()
      p.join(timeout=3)
      if p.is_alive():
          p.terminate()
      else:
          expr = d["expr"]


  if expr==None:
      return expr

  else:
    return expr

def parse_expr_timeout(string):
    from multiprocessing import Process, Manager
    from sympy.parsing.sympy_parser import parse_expr

    expr = None
    def f(d, string):
        from sympy import S; S.Half
        try:
            d['expr'] = parse_expr(string, local_dict={'x': x}, evaluate=False)
        except:
            return None

    with Manager() as manager:
        d = manager.dict()
        p = Process(target=f, args=(d, string))
        p.start()
        p.join(timeout=3)
        if p.is_alive():
            p.terminate()
        else:
            expr = d["expr"]

    return expr

In [ ]:
cd taylor/data_generation

/content/drive/MyDrive/taylor/data_generation


In [ ]:
def generate_single_sequence():
    n = random.randint(1, 3)
    root = random_binary_trees(n)
    result = traverse_unary_binary_prefix(root)
    if 'x' not in result:
        return None

    result_infix = prefix_to_infix(result)
    result_expr = " ".join(result_infix)

    #  convert to sympy expression
    result_expr = parse_expr_timeout(result_expr)
    if result_expr is None:
        return None

    #REJECT IF POLYNOMIAL
    if result_expr.is_polynomial():
      return None

    #  simplification
    result_simp = simplify_timeout(result_expr)
    if result_simp == "Error":
        return None
    
        
    #  back to prefix
    result_simp_prefix = infix_to_prefix(result_simp)
    if not result_simp_prefix:
        return None

    #  generate target
    try:
        taylor_expn =  series_timeout(result_simp)
    except ValueError:
        return None
    if taylor_expn == "Error" or None:
      return None

    expression = simplify_timeout(taylor_expn)
    
    if expression == "Error":
        return None
        
    expression_prefix = infix_to_prefix(expression)
    if not expression_prefix:
        return None

    if expression_prefix == result_simp_prefix or len(expression_prefix)>50: 
      return None

    expression_prefix = " ".join(expression_prefix)
    result_simp_prefix = " ".join(result_simp_prefix)
    return expression_prefix, result_simp_prefix


def generate_bwd(num):
    filename = "taylor_data_10000.txt"
    

    while True:
        if _FINISH:
            break
        sequence = []
        start = time.time()
        file = open(filename, "a")
        i = 1
        while i<=num:
            seq = generate_single_sequence()
            if seq is not None:
                expression_prefix, result_simp_prefix = seq
                sequence.append(result_simp_prefix + "\t" + expression_prefix + "\n")
                print((result_simp_prefix + "\t" + expression_prefix + "\n"))
                file.write(result_simp_prefix + "\t" + expression_prefix + "\n")
            else:
                continue
            i += 1

        end = time.time()
        print('process finished')
        
        file.close()
        return sequence

In [ ]:
generate_bwd(10000)

* cosh x exp 5	* / 1 24 * + 24 + pow x 4 * 12 pow x 2 exp 5

cos + -1 pow x 2	+ * pow x 2 sin 1 + * / -1 2 * pow x 4 cos 1 cos 1

sinh x	+ x * / 1 6 pow x 3

* pow x 2 asinh x	pow x 3

pow + 1 * -1 x / 1 2	+ 1 + * / -5 128 pow x 4 + * / -1 2 x + * / -1 8 pow x 2 * / -1 16 pow x 3

+ * -1 x cos + 5 x	+ * -1 x + * -1 * x sin 5 + * / -1 2 * pow x 2 cos 5 + * / 1 6 * pow x 3 sin 5 + * / 1 24 * pow x 4 cos 5 cos 5



KeyboardInterrupt: ignored

# Task-2 Processing Data for Model

In [ ]:
cd -

/content/drive/MyDrive


In [ ]:
import os
import io
import numpy as np
import re
import unicodedata
import urllib3
import shutil
import zipfile
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas
import spacy
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tqdm import tqdm_notebook
import random
from collections import Counter
import torchtext
from torchtext.data import get_tokenizer
from tqdm import tqdm
import math

In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
    
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    return w

In [ ]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [equation, series]
def create_dataset(path, num_examples=None):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [ ]:
path_to_file = "./taylor/data_generation/taylor_cleaned_pre.txt"

In [ ]:
eq, series = create_dataset(path_to_file)
print(eq[-1])
print(series[-1])

* / 1 3 * pow x -1 acos x
* pow x -1 + * / -1 3 x + * / -1 18 pow x 3 + * / -1 40 pow x 5 * / 1 6 pi


In [ ]:
X_text, Y_text = create_dataset(path_to_file)

In [ ]:
device = torch.device("cuda" if torch.torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = get_tokenizer(tokenizer = None, language = None)

In [ ]:
class FunctionDataset(torch.utils.data.Dataset):

  def __init__(self, ops=3, max_seq_length=32):

    raw_input = []
    raw_output = []

    for i in range(len(X_text)):
      expr, tay = tokenizer(X_text[i]), tokenizer(Y_text[i])

      #discards expressions too long and nan values
      if(len(expr) + 2 <= max_seq_length and len(tay) + 2 <= max_seq_length):
        #insert start and end tokens
        expr.insert(0,'<SOS>')
        expr.append('<EOS>')

        tay.insert(0,'<SOS>')
        tay.append('<EOS>')
        
        raw_input.append(expr)
        raw_output.append(tay)
        print("expression", expr)
        print("taylor series around 0", tay)
    
    #generate vocab
    self.vocab = set()
    for expr, tay in zip(raw_input, raw_output):
      self.vocab |= set(expr) |(set(tay))

    #token -> idx
    self.token_to_idx = {value : index + 1 for index, value in enumerate(self.vocab)}

    #idx -> token
    self.idx_to_token = {index + 1 : value for index, value in enumerate(self.vocab)}

    self.input = []
    self.output = []
    for raw_expr, raw_tay in zip(raw_input, raw_output):
      expr = [self.token_to_idx[token] for token in raw_expr] + [0] * (max_seq_length - len(raw_expr))
      tay = [self.token_to_idx[token] for token in raw_tay] + [0] * (max_seq_length - len(raw_tay))

      self.input.append(torch.tensor(expr, dtype=torch.long, device=device))
      self.output.append(torch.tensor(tay, dtype=torch.long, device=device))
    
  def __len__(self):
    return len(self.input)

  def __getitem__(self, idx):
    return self.input[idx].to(device), self.output[idx].to(device)

  def get_alphabet(self):
    return self.vocab

In [ ]:
d = FunctionDataset()
train_idx = list(range(0, int(9*len(d)/10)))
test_idx = list(range(int(9*len(d)/10), len(d)))
train_dataset = torch.utils.data.Subset(d, train_idx)
test_dataset = torch.utils.data.Subset(d, test_idx)

Streaming output truncated to the last 5000 lines.
expression ['<SOS>', '+', '2', '*', '/', '1', '5', 'sin', 'x', '<EOS>']
taylor series around 0 ['<SOS>', '+', '2', '+', '*', '/', '-1', '30', 'pow', 'x', '3', '*', '/', '1', '5', 'x', '<EOS>']
expression ['<SOS>', 'sinh', '*', '/', '2', '5', 'x', '<EOS>']
taylor series around 0 ['<SOS>', '*', '/', '2', '375', '*', 'x', '+', '75', '*', '2', 'pow', 'x', '2', '<EOS>']
expression ['<SOS>', '*', '-1', 'sinh', '*', 'x', 'pow', '+', '3', 'x', '-1', '<EOS>']
taylor series around 0 ['<SOS>', '*', '/', '1', '162', '*', 'x', '+', '-54', '+', '*', '-7', 'pow', 'x', '2', '+', '*', '3', 'pow', 'x', '3', '*', '18', 'x', '<EOS>']
expression ['<SOS>', '+', 'x', '*', '4', 'sin', 'x', '<EOS>']
taylor series around 0 ['<SOS>', '*', '/', '1', '3', '*', 'x', '+', '15', '*', '-2', 'pow', 'x', '2', '<EOS>']
expression ['<SOS>', '*', 'tanh', '5', 'tanh', 'x', '<EOS>']
taylor series around 0 ['<SOS>', '*', '/', '-1', '3', '*', 'x', '*', '+', '-3', 'pow', 'x', '

In [ ]:
len(train_idx)

10065

In [ ]:
len(test_idx)

1119

# LSTM Model

In [ ]:
#Defining Encoder and Decoder Class and then Model

class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim=512, num_layers=2, hidden_size=512, dropout=0.2):
    super(Encoder, self).__init__()
    self.embedding_dim = embedding_dim
    self.num_layers = num_layers
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(
        num_embeddings=vocab_size,
        embedding_dim=self.embedding_dim
    )
    self.lstm = nn.LSTM(
        input_size=self.embedding_dim,
        hidden_size=self.hidden_size,
        num_layers=self.num_layers,
        dropout=dropout,
    )
  """
  input shape (SEQUENCE_LENGTH, BATCH_SIZE)
  h,c shape (HIDDEN_SIZE)
  """
  def forward(self, x):
    embed = self.embedding(x)
    output, (h,c) = self.lstm(embed)
    return h, c

class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim=512, num_layers=2, hidden_size=512, dropout=0.2):
    super(Decoder, self).__init__()
    self.embedding_dim = embedding_dim
    self.num_layers = num_layers
    self.output_size = vocab_size
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(
        num_embeddings=vocab_size,
        embedding_dim=self.embedding_dim
    )
    self.lstm = nn.LSTM(
        input_size=self.embedding_dim,
        hidden_size=self.hidden_size,
        num_layers=self.num_layers,
        dropout=0.2,
    )
    self.out = nn.Linear(self.hidden_size, self.output_size)
    self.softmax = nn.LogSoftmax(dim=2)
    self.to(device)
  """
  input shape (BATCH_SIZE)
  output shape
  """
  def forward(self, input, h_0, c_0):
    embedded = self.embedding(input.unsqueeze(0))
    output, (h,c) = self.lstm(embedded, (h_0, c_0))
    output = self.out(output)     
    output = self.softmax(output)
    return output.squeeze(0), h , c

class Model(nn.Module):
  def __init__(self, encoder, decoder):
    super(Model, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.to(device)
  """
  Input tensor of shape (SEQUENCE_LENGTH, BATCH_SIZE)
  Output tensor of shape (SEQUENCE_LENGTH, BATCH_SIZE, VOCAB_SIZE)

  if tgt is none use teacher forecasting
  """
  def forward(self, input, tgt=None):
    if len(input.shape) < 2:
        input = input.unsqueeze(1)
    batch_size = input.shape[1]
    h, c = enc(input)
    target = torch.zeros(batch_size, dtype=torch.long).to(device)
    if tgt is None:
      max_seq_length = input.shape[0]
      target[:] = d.token_to_idx['<SOS>']
    else:
      max_seq_length = tgt.shape[1]
      target[:] = tgt[:,0]
    outputs = torch.zeros(max_seq_length, batch_size, dec.output_size, dtype=torch.float).to(device)
    for i in range(max_seq_length):
        prediction, h, c = dec(target, h, c)
        outputs[i] = prediction
        if tgt is None:
          target = prediction.argmax(dim=1)
        else:
          target = tgt[:,i]
    return outputs

In [ ]:
#Instantiate Model

enc = Encoder(len(d.get_alphabet()) + 1)
dec = Decoder(len(d.get_alphabet()) + 1)
m = Model(enc,dec).to(device)

In [ ]:
#Train and Test Epoch

def train_epoch_LSTM(model, train_loader, optimizer, criterion, batch_size=256):
  model.train()
  total_loss = 0
  total_items = 0
  num_correct = 0
  for src, tgt in tqdm(train_loader):
    src = src.to(device)
    tgt = tgt.to(device)

    pred = model(src.squeeze().T,tgt=tgt[:,:-1])

    pred = pred.permute((1,2,0))
    tgt_out = tgt[:,1:]
    loss = criterion(pred, tgt_out)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_items += (tgt_out != 0).sum(dim=(0,1))

    num_correct += (torch.logical_and((pred.argmax(dim=1) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss, num_correct / total_items

def test_epoch_LSTM(model, test_loader, criterion, batch_size=256):
  model.eval()
  total_loss = 0
  total_items = 0
  num_correct = 0
  for src, tgt in tqdm(test_loader):
    src = src.to(device)
    tgt = tgt.to(device)

    pred = model(src.squeeze().T, tgt=tgt[:,:-1])
    pred = pred.permute((1,2,0))
    tgt_out = tgt[:,1:]

    loss = criterion(pred, tgt_out)

    total_loss += loss.item()
    total_items += (tgt_out != 0).sum(dim=(0,1))

    num_correct += (torch.logical_and((logits.argmax(dim=2) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss, num_correct / total_items

In [ ]:
#Method to train LSTM Model
def train_LSTM(model, train_dataset,  test_dataset, batch_size=256, epochs=20):
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
  criterion = nn.CrossEntropyLoss()


  optim = torch.optim.Adam(model.parameters(), lr=1e-3)
  for e in range(epochs):
    train_loss, train_acc = train_epoch_LSTM(model, train_loader, optim, criterion, batch_size=batch_size)
    test_loss, test_acc = train_epoch_LSTM(model, test_loader, optim, criterion, batch_size=batch_size)
    print(f'Epoch: {e + 1} Training Loss: {train_loss} Training Accuracy: {train_acc} Test Loss: {test_loss} Test Accuracy: {test_acc}')

In [ ]:
#Running training
train_LSTM(m, train_dataset, test_dataset, batch_size=256)

100%|██████████| 5/5 [00:00<00:00,  7.63it/s]


Epoch: 1 Training Loss: 79.44888865947723 Training Accuracy: 0.2158389538526535 Test Loss: 6.193756699562073 Test Accuracy: 0.42160460352897644


100%|██████████| 5/5 [00:00<00:00,  7.71it/s]


Epoch: 2 Training Loss: 40.253355503082275 Training Accuracy: 0.5204954743385315 Test Loss: 4.453214883804321 Test Accuracy: 0.5615102052688599


100%|██████████| 5/5 [00:00<00:00,  7.50it/s]


...

Epoch: 18 Training Loss: 6.802447006106377 Training Accuracy: 0.9074389338493347 Test Loss: 0.8278125524520874 Test Accuracy: 0.912322998046875


100%|██████████| 5/5 [00:00<00:00,  7.35it/s]


Epoch: 19 Training Loss: 6.1967692375183105 Training Accuracy: 0.9162402749061584 Test Loss: 0.7322542071342468 Test Accuracy: 0.9216046333312988


100%|██████████| 5/5 [00:00<00:00,  7.38it/s]

Epoch: 20 Training Loss: 5.650625139474869 Training Accuracy: 0.9244372844696045 Test Loss: 0.6881309002637863 Test Accuracy: 0.9255374670028687


# Transformer Model

In [ ]:
#Defining Transformer Model

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

  src_padding_mask = (src == 0).transpose(0, 1)
  tgt_padding_mask = (tgt == 0).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

class TransformerModel(nn.Module):
    def __init__(self, num_encoder_layers, nhead, num_decoder_layers,
                 emb_size, src_vocab_size, tgt_vocab_size,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(TransformerModel, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_size, nhead=nhead,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = nn.TransformerDecoderLayer(d_model=emb_size, nhead=nhead,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.emb_size = emb_size
        self.src_tok_emb = self.embedding = nn.Embedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = self.embedding = nn.Embedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src, trg, src_mask,
                tgt_mask, src_padding_mask,
                tgt_padding_mask, memory_key_padding_mask):
        src_emb = self.positional_encoding(self.src_tok_emb(src)* math.sqrt(self.emb_size))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg)* math.sqrt(self.emb_size))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

In [ ]:
#Instantiate model
model = TransformerModel(num_encoder_layers=6, nhead=8, num_decoder_layers=6,
                 emb_size=512, src_vocab_size=(len(d.get_alphabet()) + 1), tgt_vocab_size=(len(d.get_alphabet()) + 1),
                 dim_feedforward = 512, dropout = 0.2).to(device)

In [ ]:
#Train and Test Epoch

def train_epoch_transformer(model, train_loader, optimizer, criterion, batch_size):
  model.train()
  total_loss = 0
  num_correct = 0
  total_items = 0
  for src, tgt in tqdm(train_loader):
      src = src.to(device).T
      tgt = tgt.to(device).T

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)

      optimizer.zero_grad()

      tgt_out = tgt[1:,:]
      loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))

      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      total_items += (tgt_out != 0).sum(dim=(0,1))

      num_correct += (torch.logical_and((logits.argmax(dim=2) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss / len(train_loader), num_correct / total_items


def test_epoch_transformer(model, test_loader, criterion, batch_size):
  model.eval()
  total_loss = 0
  num_correct = 0
  total_items = 0
  for src, tgt in tqdm(train_loader):
      src = src.to(device).T
      tgt = tgt.to(device).T

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)


      tgt_out = tgt[1:,:]
      loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))


      total_loss += loss.item()
      total_items += (tgt_out != 0).sum(dim=(0,1))

      num_correct += (torch.logical_and((logits.argmax(dim=2) == tgt_out), (tgt_out != 0))).sum(dim=(0,1))
  return total_loss / len(train_loader), num_correct / total_items

In [ ]:
#Defining Transformer Training Method

def train_transformer(model, train_dataset, test_dataset, batch_size=256, epochs=40):
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
  criterion = nn.CrossEntropyLoss()
  
  optim = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
  for e in range(epochs):
    train_loss, train_acc = train_epoch_transformer(model, train_loader, optim, criterion, batch_size=batch_size)
    test_loss, test_acc = train_epoch_transformer(model, test_loader, optim, criterion, batch_size=batch_size)
    print(f'Epoch: {e + 1} Training Loss: {train_loss} Training Accuracy: {train_acc} Test Loss: {test_loss} Test Accuracy: {test_acc}')

In [ ]:
#Running Training 

train_transformer(model,train_dataset, test_dataset)

  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and src_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 5/5 [00:02<00:00,  2.37it/s]


Epoch: 1 Training Loss: 2.272262266278267 Training Accuracy: 0.16307766735553741 Test Loss: 1.2473761320114136 Test Accuracy: 0.46979549527168274


100%|██████████| 5/5 [00:02<00:00,  2.37it/s]


Epoch: 2 Training Loss: 1.017479281127453 Training Accuracy: 0.5379163026809692 Test Loss: 0.8975930333137512 Test Accuracy: 0.5847928524017334


100%|██████████| 5/5 [00:02<00:00,  2.36it/s]


...

Epoch: 38 Training Loss: 0.15924151353538035 Training Accuracy: 0.8997758626937866 Test Loss: 0.1581922322511673 Test Accuracy: 0.8990036845207214


100%|██████████| 5/5 [00:02<00:00,  2.36it/s]


Epoch: 39 Training Loss: 0.15672463215887547 Training Accuracy: 0.9010021686553955 Test Loss: 0.15758674442768097 Test Accuracy: 0.9023597240447998


100%|██████████| 5/5 [00:02<00:00,  2.37it/s]

Epoch: 40 Training Loss: 0.15127565003931523 Training Accuracy: 0.904153048992157 Test Loss: 0.15667804181575776 Test Accuracy: 0.9035658240318298
